### 顔認識ニューラルネットワーク用データセット生成プログラム

In [25]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import xml.etree.ElementTree as ET
import os
import os.path
import shutil

#### 元画像から一人ずつ顔を切り出す

In [26]:
data_list = open("/home/alphabet0-9/data_list.txt")
# データリスト-----------------------------------------------------------
# (中身)
# /home/alphabet0-9/label_10_p17_p24_5
# /home/alphabet0-9/label_10_p1_p8_5
# /home/alphabet0-9/label_10_p9_p16_5
# ・・・
#
# ファイル名の拡張子手前まで記述しているリストを読み取る
# ------------------------------------------------------------------------

# ファイル分類用連想配列の初期化------------------------------------------
# key : クラス名
# value : 枚数
# ------------------------------------------------------------------------
count_dict = {
    '-':0,
    '0':0,'1':0,'2':0,'3':0,'4':0,'5':0,'6':0,'7':0,'8':0,'9':0,
    'A':0,'B':0,'C':0,'D':0,'E':0,'F':0,'G':0,'H':0,'I':0,'J':0,'K':0,'L':0,'M':0,
    'N':0,'O':0,'P':0,'Q':0,'R':0,'S':0,'T':0,'U':0,'V':0,'W':0,'X':0,'Y':0,'Z':0
}

# ラベルリストを参照し繰り返す--------------------------------------------
for line in data_list:
    # 改行文字の置換------------------------------------------------------
    line = line.replace('\n','')
    
    # ファイル指定--------------------------------------------------------
    img_file = line+".jpg"
    annotation_file = line+".xml"
    
    # 画像の読み込み------------------------------------------------------
    image = cv2.imread(img_file)
    
    # xmlファイルから座標を取得-------------------------------------------    
    xml = open(annotation_file)
    tree = ET.parse(xml)
    root = tree.getroot()
    
    xy_list = [] #　座標リスト
    
    for obj in root.iter('object') :
        # 座標リストの格納-------------------------------------------------
        # cls : クラス名
        # b : 座標のリスト
        # -----------------------------------------------------------------
        cls = obj.find('name').text
        xmlbox = obj.find('bndbox')
        b = [int(xmlbox.find('xmin').text), 
             int(xmlbox.find('ymin').text), 
             int(xmlbox.find('xmax').text), 
             int(xmlbox.find('ymax').text)]
        xy_list.append(b + [cls])
    
    # 座標リストから画像の生成---------------------------------------------
    for i in range(len(xy_list)) :
        
        # 矩形領域の座標指定-----------------------------------------------
        ymin = xy_list[i][1]
        ymax = xy_list[i][3]
        xmin = xy_list[i][0]
        xmax = xy_list[i][2]
        
        # クラス名の指定---------------------------------------------------
        name = xy_list[i][4]
        
        # 領域部分を切り取る-----------------------------------------------
        split_image = image[ymin:ymax,xmin:xmax]
        split_image = cv2.resize(split_image,(28,28))
        
        # クラスのディレクトリが無ければ生成-------------------------------
        dir_name= "/home/alphabet0-9/data/" + name +"/"
        
        if os.path.isdir(dir_name) == False :
            os.mkdir(dir_name)
        
        # 画像の書き込み----------------------------------------------------
        file_name = dir_name + str(count_dict[name]) + ".jpg"
        cv2.imwrite(file_name,split_image)
        
        # 枚数のカウント-----------------------------------------------------
        count_dict[name] += 1

/home/alphabet0-9/label_10_p17_p24_5 313
/home/alphabet0-9/label_10_p1_p8_5 2379
/home/alphabet0-9/label_10_p9_p16_5 1008
/home/alphabet0-9/label_10_p65_p72_5 1274
/home/alphabet0-9/label_10_p49_p56_5 1041
/home/alphabet0-9/label_11_p1_p8_5 1279


In [27]:
total = 0 # 合計枚数

# 各クラスの画像枚数の出力 --------------------------------------------------
for key, value in count_dict.items():
    print("[" + key + "]",value,"枚")
    total += value

# 全クラスの合計枚数の出力---------------------------------------------------
print("現在 "+str(total) + " 枚")

[0 : 0] 185 枚
[1 : 1] 194 枚
[2 : 2] 174 枚
[3 : 3] 136 枚
[4 : 4] 175 枚
[5 : 5] 146 枚
[6 : 6] 168 枚
[7 : 7] 219 枚
[8 : 8] 195 枚
[9 : 9] 150 枚
[10 : A] 224 枚
[11 : B] 210 枚
[12 : C] 203 枚
[13 : D] 216 枚
[14 : E] 195 枚
[15 : F] 200 枚
[16 : G] 221 枚
[17 : H] 203 枚
[18 : I] 226 枚
[19 : J] 196 枚
[20 : K] 199 枚
[21 : L] 194 枚
[22 : M] 209 枚
[23 : N] 205 枚
[24 : O] 210 枚
[25 : P] 198 枚
[26 : Q] 234 枚
[27 : R] 202 枚
[28 : S] 215 枚
[29 : T] 232 枚
[30 : U] 185 枚
[31 : V] 192 枚
[32 : W] 205 枚
[33 : X] 198 枚
[34 : Y] 197 枚
[35 : Z] 197 枚
[36 : -] 186 枚
現在 7294 枚


#### TrainとValidationファイルに分類

In [28]:
# ディレクトリ指定-----------------------------------------------------------
# dir_name : 元画像のディレクトリ
# train_dir : 訓練用データセットのディレクトリ
# validation_dir : 検証用データセットのディレクトリ
# ----------------------------------------------------------------------------
dir_name = "/home/alphabet0-9/data/"
train_dir = "/home/alphabet0-9/train/"
validation_dir = "/home/alphabet0-9/validation/"

# ディレクトリがなければ生成する----------------------------------------------
if os.path.isdir(train_dir) == False:
    os.mkdir(train_dir)
if os.path.isdir(validation_dir) == False:
    os.mkdir(validation_dir)

# 元画像のディレクトリを参照し繰り返す----------------------------------------
for root, dirs, files in os.walk(dir_name):
    for d in dirs :
        # 走査するディレクトリの指定------------------------------------------
        scanning_dir_name = dir_name+d+"/"
        for root,dirs,files in os.walk(scanning_dir_name) :
            i = 0 # 枚数カウント用
            
            # 画像ファイルを参照し繰り返す------------------------------------
            for f in files :
                # 全体の75%はTrain，残りの25%はValidationに分類---------------
                if i/len(files) < 0.75 :
                    mode = "train/"
                else :
                    mode = "validation/"
                
                # 訓練・検証ディレクトリにクラスディレクトリを生成------------
                make_dir_name = "/home/alphabet0-9/" + mode + d + "/" 
                if os.path.isdir(make_dir_name) == False:
                    os.mkdir(make_dir_name)
                    
                # 生成したディレクトリに元画像をコピー------------------------
                file_name = make_dir_name + f
                shutil.copyfile(scanning_dir_name + f, file_name)
                
                # 枚数カウントアップ-------------------------------------------
                i += 1 

/home/alphabet0-9/data/L/71.jpg => /home/alphabet0-9/train/L/71.jpg
/home/alphabet0-9/data/L/136.jpg => /home/alphabet0-9/train/L/136.jpg
/home/alphabet0-9/data/L/39.jpg => /home/alphabet0-9/train/L/39.jpg
/home/alphabet0-9/data/L/56.jpg => /home/alphabet0-9/train/L/56.jpg
/home/alphabet0-9/data/L/89.jpg => /home/alphabet0-9/train/L/89.jpg
/home/alphabet0-9/data/L/86.jpg => /home/alphabet0-9/train/L/86.jpg
/home/alphabet0-9/data/L/102.jpg => /home/alphabet0-9/train/L/102.jpg
/home/alphabet0-9/data/L/72.jpg => /home/alphabet0-9/train/L/72.jpg
/home/alphabet0-9/data/L/105.jpg => /home/alphabet0-9/train/L/105.jpg
/home/alphabet0-9/data/L/94.jpg => /home/alphabet0-9/train/L/94.jpg
/home/alphabet0-9/data/L/5.jpg => /home/alphabet0-9/train/L/5.jpg
/home/alphabet0-9/data/L/78.jpg => /home/alphabet0-9/train/L/78.jpg
/home/alphabet0-9/data/L/6.jpg => /home/alphabet0-9/train/L/6.jpg
/home/alphabet0-9/data/L/168.jpg => /home/alphabet0-9/train/L/168.jpg
/home/alphabet0-9/data/L/150.jpg => /home/al

/home/alphabet0-9/data/N/201.jpg => /home/alphabet0-9/train/N/201.jpg
/home/alphabet0-9/data/N/48.jpg => /home/alphabet0-9/train/N/48.jpg
/home/alphabet0-9/data/N/159.jpg => /home/alphabet0-9/train/N/159.jpg
/home/alphabet0-9/data/N/123.jpg => /home/alphabet0-9/train/N/123.jpg
/home/alphabet0-9/data/N/47.jpg => /home/alphabet0-9/train/N/47.jpg
/home/alphabet0-9/data/N/18.jpg => /home/alphabet0-9/train/N/18.jpg
/home/alphabet0-9/data/N/170.jpg => /home/alphabet0-9/train/N/170.jpg
/home/alphabet0-9/data/N/46.jpg => /home/alphabet0-9/train/N/46.jpg
/home/alphabet0-9/data/N/149.jpg => /home/alphabet0-9/train/N/149.jpg
/home/alphabet0-9/data/N/121.jpg => /home/alphabet0-9/train/N/121.jpg
/home/alphabet0-9/data/N/164.jpg => /home/alphabet0-9/train/N/164.jpg
/home/alphabet0-9/data/N/175.jpg => /home/alphabet0-9/train/N/175.jpg
/home/alphabet0-9/data/N/32.jpg => /home/alphabet0-9/train/N/32.jpg
/home/alphabet0-9/data/N/133.jpg => /home/alphabet0-9/train/N/133.jpg
/home/alphabet0-9/data/N/125.j

/home/alphabet0-9/data/O/164.jpg => /home/alphabet0-9/train/O/164.jpg
/home/alphabet0-9/data/O/175.jpg => /home/alphabet0-9/train/O/175.jpg
/home/alphabet0-9/data/O/32.jpg => /home/alphabet0-9/train/O/32.jpg
/home/alphabet0-9/data/O/133.jpg => /home/alphabet0-9/train/O/133.jpg
/home/alphabet0-9/data/O/125.jpg => /home/alphabet0-9/train/O/125.jpg
/home/alphabet0-9/data/O/178.jpg => /home/alphabet0-9/train/O/178.jpg
/home/alphabet0-9/data/O/154.jpg => /home/alphabet0-9/train/O/154.jpg
/home/alphabet0-9/data/O/80.jpg => /home/alphabet0-9/train/O/80.jpg
/home/alphabet0-9/data/O/50.jpg => /home/alphabet0-9/train/O/50.jpg
/home/alphabet0-9/data/O/45.jpg => /home/alphabet0-9/train/O/45.jpg
/home/alphabet0-9/data/O/139.jpg => /home/alphabet0-9/train/O/139.jpg
/home/alphabet0-9/data/O/33.jpg => /home/alphabet0-9/train/O/33.jpg
/home/alphabet0-9/data/O/120.jpg => /home/alphabet0-9/train/O/120.jpg
/home/alphabet0-9/data/O/51.jpg => /home/alphabet0-9/train/O/51.jpg
/home/alphabet0-9/data/O/141.jpg

/home/alphabet0-9/data/Q/49.jpg => /home/alphabet0-9/train/Q/49.jpg
/home/alphabet0-9/data/Q/104.jpg => /home/alphabet0-9/train/Q/104.jpg
/home/alphabet0-9/data/Q/225.jpg => /home/alphabet0-9/train/Q/225.jpg
/home/alphabet0-9/data/Q/4.jpg => /home/alphabet0-9/train/Q/4.jpg
/home/alphabet0-9/data/Q/180.jpg => /home/alphabet0-9/train/Q/180.jpg
/home/alphabet0-9/data/Q/3.jpg => /home/alphabet0-9/train/Q/3.jpg
/home/alphabet0-9/data/Q/187.jpg => /home/alphabet0-9/train/Q/187.jpg
/home/alphabet0-9/data/Q/113.jpg => /home/alphabet0-9/train/Q/113.jpg
/home/alphabet0-9/data/Q/29.jpg => /home/alphabet0-9/train/Q/29.jpg
/home/alphabet0-9/data/Q/212.jpg => /home/alphabet0-9/train/Q/212.jpg
/home/alphabet0-9/data/Q/90.jpg => /home/alphabet0-9/train/Q/90.jpg
/home/alphabet0-9/data/Q/97.jpg => /home/alphabet0-9/train/Q/97.jpg
/home/alphabet0-9/data/Q/182.jpg => /home/alphabet0-9/train/Q/182.jpg
/home/alphabet0-9/data/Q/43.jpg => /home/alphabet0-9/train/Q/43.jpg
/home/alphabet0-9/data/Q/111.jpg => /h

/home/alphabet0-9/data/G/5.jpg => /home/alphabet0-9/train/G/5.jpg
/home/alphabet0-9/data/G/78.jpg => /home/alphabet0-9/train/G/78.jpg
/home/alphabet0-9/data/G/6.jpg => /home/alphabet0-9/train/G/6.jpg
/home/alphabet0-9/data/G/168.jpg => /home/alphabet0-9/train/G/168.jpg
/home/alphabet0-9/data/G/150.jpg => /home/alphabet0-9/train/G/150.jpg
/home/alphabet0-9/data/G/52.jpg => /home/alphabet0-9/train/G/52.jpg
/home/alphabet0-9/data/G/181.jpg => /home/alphabet0-9/train/G/181.jpg
/home/alphabet0-9/data/G/137.jpg => /home/alphabet0-9/train/G/137.jpg
/home/alphabet0-9/data/G/156.jpg => /home/alphabet0-9/train/G/156.jpg
/home/alphabet0-9/data/G/115.jpg => /home/alphabet0-9/train/G/115.jpg
/home/alphabet0-9/data/G/42.jpg => /home/alphabet0-9/train/G/42.jpg
/home/alphabet0-9/data/G/58.jpg => /home/alphabet0-9/train/G/58.jpg
/home/alphabet0-9/data/G/146.jpg => /home/alphabet0-9/train/G/146.jpg
/home/alphabet0-9/data/G/166.jpg => /home/alphabet0-9/train/G/166.jpg
/home/alphabet0-9/data/G/220.jpg => 

/home/alphabet0-9/data/B/9.jpg => /home/alphabet0-9/train/B/9.jpg
/home/alphabet0-9/data/B/124.jpg => /home/alphabet0-9/train/B/124.jpg
/home/alphabet0-9/data/B/1.jpg => /home/alphabet0-9/train/B/1.jpg
/home/alphabet0-9/data/B/74.jpg => /home/alphabet0-9/train/B/74.jpg
/home/alphabet0-9/data/B/101.jpg => /home/alphabet0-9/train/B/101.jpg
/home/alphabet0-9/data/B/54.jpg => /home/alphabet0-9/train/B/54.jpg
/home/alphabet0-9/data/B/36.jpg => /home/alphabet0-9/train/B/36.jpg
/home/alphabet0-9/data/B/169.jpg => /home/alphabet0-9/train/B/169.jpg
/home/alphabet0-9/data/B/206.jpg => /home/alphabet0-9/train/B/206.jpg
/home/alphabet0-9/data/B/79.jpg => /home/alphabet0-9/train/B/79.jpg
/home/alphabet0-9/data/B/16.jpg => /home/alphabet0-9/train/B/16.jpg
/home/alphabet0-9/data/B/171.jpg => /home/alphabet0-9/train/B/171.jpg
/home/alphabet0-9/data/B/194.jpg => /home/alphabet0-9/train/B/194.jpg
/home/alphabet0-9/data/B/142.jpg => /home/alphabet0-9/train/B/142.jpg
/home/alphabet0-9/data/B/28.jpg => /ho

/home/alphabet0-9/data/R/31.jpg => /home/alphabet0-9/validation/R/31.jpg
/home/alphabet0-9/data/R/117.jpg => /home/alphabet0-9/validation/R/117.jpg
/home/alphabet0-9/data/R/196.jpg => /home/alphabet0-9/validation/R/196.jpg
/home/alphabet0-9/data/R/195.jpg => /home/alphabet0-9/validation/R/195.jpg
/home/alphabet0-9/data/R/193.jpg => /home/alphabet0-9/validation/R/193.jpg
/home/alphabet0-9/data/R/189.jpg => /home/alphabet0-9/validation/R/189.jpg
/home/alphabet0-9/data/R/128.jpg => /home/alphabet0-9/validation/R/128.jpg
/home/alphabet0-9/data/R/144.jpg => /home/alphabet0-9/validation/R/144.jpg
/home/alphabet0-9/data/R/76.jpg => /home/alphabet0-9/validation/R/76.jpg
/home/alphabet0-9/data/R/25.jpg => /home/alphabet0-9/validation/R/25.jpg
/home/alphabet0-9/data/R/24.jpg => /home/alphabet0-9/validation/R/24.jpg
/home/alphabet0-9/data/R/27.jpg => /home/alphabet0-9/validation/R/27.jpg
/home/alphabet0-9/data/R/109.jpg => /home/alphabet0-9/validation/R/109.jpg
/home/alphabet0-9/data/R/183.jpg =>

/home/alphabet0-9/data/T/54.jpg => /home/alphabet0-9/train/T/54.jpg
/home/alphabet0-9/data/T/36.jpg => /home/alphabet0-9/train/T/36.jpg
/home/alphabet0-9/data/T/169.jpg => /home/alphabet0-9/train/T/169.jpg
/home/alphabet0-9/data/T/206.jpg => /home/alphabet0-9/train/T/206.jpg
/home/alphabet0-9/data/T/79.jpg => /home/alphabet0-9/train/T/79.jpg
/home/alphabet0-9/data/T/16.jpg => /home/alphabet0-9/train/T/16.jpg
/home/alphabet0-9/data/T/171.jpg => /home/alphabet0-9/train/T/171.jpg
/home/alphabet0-9/data/T/211.jpg => /home/alphabet0-9/train/T/211.jpg
/home/alphabet0-9/data/T/194.jpg => /home/alphabet0-9/train/T/194.jpg
/home/alphabet0-9/data/T/142.jpg => /home/alphabet0-9/train/T/142.jpg
/home/alphabet0-9/data/T/28.jpg => /home/alphabet0-9/train/T/28.jpg
/home/alphabet0-9/data/T/30.jpg => /home/alphabet0-9/train/T/30.jpg
/home/alphabet0-9/data/T/73.jpg => /home/alphabet0-9/train/T/73.jpg
/home/alphabet0-9/data/T/106.jpg => /home/alphabet0-9/train/T/106.jpg
/home/alphabet0-9/data/T/122.jpg =